In [1]:
# import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as _sum

In [ ]:
spark = SparkSession.builder \
    .appName("CumulativeSumExample") \
    .getOrCreate()

In [ ]:
dataset=pd.read_csv('filtered_ShubmanGill.csv')

In [ ]:
numeric_columns=['inns_number','ball','outs','over','noball','wide','year','runs','balls','outs','dots','1s','2s','3s','4s','5s','6s','control','bowler_dismissed?','dots']
for i in numeric_columns:
    dataset[i].astype(int)
    
dataset['ball_id'].astype(float)

In [ ]:
dataset['date']=pd.to_datetime(dataset['date'])

In [ ]:
dataset=dataset.sort_values(by=['date','ball_id'])

In [ ]:
dataset.groupby(['date','over','ball']).cumsum()

In [ ]:
numeric_cols=dataset.groupby(['date'])['runs','balls','outs','dots','1s','2s','3s','4s','5s','6s','control'].cumsum()
nonnumeric_cols=dataset[['date','team','inns_number','opposition','ground','dismissal']]

In [ ]:
cumulativedataset=pd.concat([nonnumeric_cols,numeric_cols],axis=1)

In [ ]:
cumulativedataset

In [ ]:
cumulativedataset['SR']=round(cumulativedataset['runs']*100/cumulativedataset['balls'],2)
cumulativedataset['Control%']=round(cumulativedataset['control']*100/cumulativedataset['balls'],2)
cumulativedataset.drop(['control'],axis=1,inplace=True)

In [ ]:
transformdataset=cumulativedataset[['date','team','inns_number','opposition','ground','runs','balls','outs']]

In [ ]:
transformdataset.set_index(['balls'],inplace=True)
transformdataset

In [ ]:
transformdataset=transformdataset.T
transformdataset

In [ ]:
grouped_df = cumulativedataset.groupby(['date','team','inns_number','opposition','ground','balls'])['runs'].sum().reset_index()
pivot_df = grouped_df.pivot(index='date', columns='balls', values='runs')
pivot_df = pivot_df.reindex(sorted(pivot_df.columns), axis=1)

In [ ]:
pivot_df

In [ ]:
grouped_df.head(56
            )